## Load libraries

In [ ]:
library(limma)
library(statmod)
library(ggplot2)
library(dplyr)
library(tidyverse)
library(edgeR)
library(data.table)
library(readr)
library(tibble)
library(reshape2)
library(pheatmap)
library(yaml)
library(stringr)
library(ggh4x)
library(matrixStats)
library(RColorBrewer)
library(DESeq2)

## Load data

In [2]:
counts <- read.table("counts_matrix.txt", sep = '\t', header = TRUE)
rownames(counts) <- counts$GENE
counts <- counts[-1]
colnames(counts) <- gsub("\\.", "-", colnames(counts))
head(counts)

,CW20012_NRXN1_Het_a,CW50037_NRXN1_Homo_b,CW50037_NRXN1_Homo_c,CW50037_NRXN1_Homo_d,CW20012_NRXN1_Het_b,CW20012_NRXN1_Het_c,CW20012_NRXN1_Het_d,ML898-4425_NRXN1_Homo_a,ML898-4425_NRXN1_Homo_b,ML898-4425_NRXN1_Homo_c,...,ML909-6344_NRXN1_WT_d,ML911-9779_NRXN1_Homo_a,ML911-9779_NRXN1_Homo_b,ML911-9779_NRXN1_Homo_c,ML911-9779_NRXN1_Homo_d,CW20103_NRXN1_Homo_a,CW20012_NRXN1_Homo_a,CW20012_NRXN1_Homo_b,CW20012_NRXN1_Homo_c,CW20012_NRXN1_Homo_d
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5_8S_rRNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,0,1,0,0
7SK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1BG,86,14,22,13,38,26,28,25,4,33,...,1,11,9,0,14,13,34,19,28,26
A1BG-AS1,69,21,12,13,10,12,7,8,2,4,...,1,2,1,3,7,1,18,10,24,24
A1CF,4,1,1,2,1,0,0,0,1,0,...,1,1,1,0,0,0,0,0,4,0
A2M,2602,507,345,300,1417,1547,1415,498,1,278,...,25,94,282,84,676,276,1913,1350,1420,2105


In [3]:
#Remove relevant X and Y-linked genes
sex_genes <- c("DBY", "SMCY", "UTY", "RPS4Y", "USP9Y", "XIST")
counts<- counts[!rownames(counts) %in% sex_genes, ]
dim(counts)

[1] 37201   119

In [5]:
counts[is.na(counts)] <- 0

In [ ]:
#Make metadata table

metadata <- data.frame(sample=colnames(counts)) 
metadata$gene <- str_to_upper(str_split_fixed(colnames(counts),"_",n=4)[,2])
metadata$donor <- str_to_upper(str_split_fixed(colnames(counts),"_",n=4)[,1])
metadata$genotype <- str_to_upper(str_split_fixed(colnames(counts),"_",n=4)[,3])
metadata$target_gt <- paste0(metadata$gene,"_",metadata$genotype)
head(metadata)

In [ ]:
raw_metadata <- read.table("/path_to_metadata.tsv",sep="\t",header=T)
metadata$PRS <- raw_metadata[match(metadata$donor,raw_metadata$Donor),"PRS"]
metadata$sex <- raw_metadata[match(metadata$donor,raw_metadata$Donor),"Sex"]
rownames(metadata) <- NULL
head(metadata)

In [8]:
names(counts) <- sub("\\-","",names(counts))
metadata$donor <- sub("\\-","",metadata$donor)
metadata$sample <- sub("\\-","",metadata$sample)
metadata[is.na(metadata$PRS),"PRS"] <- "NEUTRAL"
head(metadata)

In [ ]:
#Plot library size

get_library_size <- function(count_data) {
  df <- as.data.frame(count_data)
  
  long_df <- df %>% 
    pivot_longer(cols = everything(), names_to = "sample", values_to = "reads")
  
  #Remove replicate indicators
  long_df$sample <- gsub("_a|_b|_c|_d|_e|_f$", "", long_df$sample)
  

  aggregated_df <- long_df %>% 
    group_by(sample) %>%
    summarise(total_reads = sum(reads, na.rm = TRUE))

  aggregated_df$donor <- sub("_.*", "", aggregated_df$sample)
  aggregated_df$genotype <- sub(".*_", "", aggregated_df$sample)


  
  return(aggregated_df)
}

#Library size bar graph

lib_size_bar <- function(library_data) {
  if (!"donor" %in% names(library_data)) {
    library_data$donor <- sub("_.*", "", library_data$sample)
    library_data$genotype <- sub(".*_(.*)$", "\\1", library_data$sample)
  }

  library_data$donor_genotype <- with(library_data, paste(donor, genotype, sep = "_"))

  genotype_order <- c("WT", "Het", "Homo")
  
  library_data <- library_data %>%
    mutate(
      donor_genotype = factor(donor_genotype, levels = unique(donor_genotype[order(donor, match(genotype, genotype_order))]))
    )


  lib_size <- ggplot(library_data, aes(x = donor_genotype, y = total_reads, fill = donor)) +
    geom_bar(stat = "identity") +
    labs(title = "Library Size per Sample", x = "Sample", y = "Counts") +
    scale_fill_brewer(palette = "Set3", name = "Donor") +
    scale_x_discrete(labels = function(x) stringr::str_wrap(x, width = 10)) +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, size = 8))

  return(lib_size)
}

library_data <- get_library_size(counts)
lib_bar <- lib_size_bar(library_data)
lib_bar

# Run differential expression

In [12]:
make_donor_contrasts <- function(design,dge) {

    #get one contrast
    getOneContrast <- function(idx, dge, samples) {
        d <- dge$samples[idx,]$donor
        controls <- samples[grep(paste0(d,"NRXN1_WT"), samples)] #use NRXN1_WT as control
        treats <- dge$samples[idx,]$group
        if (length(treats)>0 & length(controls)>0 & (!treats %like% "NRXN1_WT")) {
            comparison <- paste(treats,controls,sep=" - ")
            names(comparison) <- treats
            return (comparison)
        } else {
            print("No valid case/control group -- skipping contrast")
            return(NULL)
        }
    }
    samples <- colnames(design)
    r=c()
    for (i in seq_len(dim(dge$samples)[1])) {
        r <- c(r,getOneContrast(i,dge,samples))
    }
    r = as.list(r)
    r=c(r,list(levels=design))
    r=r[!duplicated(r)]
    contr.matrix=do.call(makeContrasts,r)
    return (contr.matrix)
}

run_DGEA <- function(counts, metadata) { 
    # initialize DGE object + metadata
    print("Initializing DGE object...")
    dge <- DGEList(counts=counts) #create DGEList object
    dge$samples$target_gt <- factor(metadata$target_gt[match(rownames(dge$samples),metadata$sample)],levels=c("PARENTAL_WT","NRXN1_WT","NRXN1_HET","NRXN1_HOMO","LRP1_WT","NRXN1_HET","NRXN1_HOMO"))
    dge$samples$PRS <- factor(metadata$PRS[match(rownames(dge$samples),metadata$sample)],levels=c("NONE","LOW","NEUTRAL","HIGH"))
    
    dge$samples$group <- paste0(dge$samples$donor,".",dge$samples$target_gt)

    dge <- dge[,! dge$samples$target_gt %like% "NRXN11"]
    
    print("Filtering DGE...")
    # filter + normalize DGE
    keep.exprs <- filterByExpr(dge, group=dge$samples$group)
    dge <- dge[keep.exprs,, keep.lib.sizes=FALSE]

    # filter out samples with shit library sizes
    dge <- dge[,dge$samples$lib.size>100000]
    # keep <- rowSums(cpm(dge)>=50)>=4
    # print(paste0("Keeping ",sum(keep)," genes..."))
    # dge <- filterDGE(dge,keep)
    ID <- rownames(dge)
    dge$genes <- as.data.frame(ID)
    metadata <- subset(metadata, sample %in% rownames(dge$samples))

    #check ratio of largest to smallest library size
    #this helps us determine whether to use limma-trend or voom
    print(max(colSums(dge$counts))/min(colSums(dge$counts))) #if >>3, use voom. otherwise, use limma-trend

    print("Normalizing DGE...")
    dge <- calcNormFactors(dge) #calculate normalization factors; used for voom transform later

    print("Initializing design and contrast matrices...")
    # initialize design matrix for differential expression
    #design <- model.matrix(~ 0 + target_gt + sex, data=dge$samples)
    design <- model.matrix(~ 0 + target_gt, data=dge$samples)

    design <- design[, colSums(design != 0) > 0]
    colnames(design) <- sub("target_gt","",colnames(design))
    colnames(design) <- sub("PRS","",colnames(design))
    colnames(design) <- sub("donor","",colnames(design))
    colnames(design) <- sub("group","",colnames(design))
    colnames(design) <- sub("sex","",colnames(design))
    colnames(design) <- sub("\\s","",colnames(design))
    colnames(design) <- sub(":",".",colnames(design))

    #select contrast
    contr.matrix=makeContrasts(
                        #NRXN1.HOMO=NRXN1_HOMO-NRXN1_WT,
                        #NRXN1.HET=NRXN1_HET-NRXN1_WT,
                        #NRXN1_HOMO-NRXN1_HET,
                        #NRXN1.WT=NRXN1_WT-PARENTAL_WT,
                        #HIGH_LOW = HIGH - LOW,
                        #HIGH_NEU = HIGH - NEUTRAL,
                        #NEU_LOW = NEUTRAL - LOW,
                        NRXN1.HOMO=LRP1_HOMO-LRP1_WT,
                        LRP1.HET=LRP1_HET-LRP1_WT,
                        LRP1_HOMO-LRP1_HET,
                        #LRP1.WT=LRP1_WT-PARENTAL_WT,
                        levels=colnames(design))


    print("Performing differential expression fit with voom...")
    # perform differential expression fit
    v <- voom(dge,design,plot=T)
    colnames(v$genes) <- "ID"


    fit <- lmFit(v,design)
    fit <- contrasts.fit(fit,contr.matrix)
    fit <- eBayes(fit) 
    plotSA(fit) 

    return(fit)
    print("Done!")
}

In [13]:
metadata$target_gt <- factor(metadata$target_gt, levels = c("NRXN1_WT", "NRXN1_HET", "NRXN1_HOMO"))
metadata$sex <- factor(metadata$sex, levels = c("Male", "Female"))
metadata$PRS <- factor(metadata$PRS, levels = c("LOW", "NEUTRAL", "HIGH"))


In [ ]:
fit <- run_DGEA(counts,metadata)

In [ ]:
#dds <- DESeqDataSetFromMatrix(countData = counts, colData = metadata, design = ~ 0 + target_gt + sex)
dds <- DESeqDataSetFromMatrix(countData = counts, colData = metadata, design = ~ 0 + target_gt)

vsd <- vst(dds, blind=FALSE)
pcaData <- plotPCA(vsd, intgroup = c("target_gt"), returnData = TRUE) 
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=target_gt)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ", percentVar[1], "% variance")) +
  ylab(paste0("PC2: ", percentVar[2], "% variance")) +
  coord_fixed() +
  theme(
    panel.grid.major = element_blank(),  
    panel.grid.minor = element_blank(), 
    panel.background = element_blank(),  
    axis.line = element_line(color = "black")  
  )

#Run DE analysis
dds <- DESeq(dds)
resultsNames(dds) 


In [ ]:
#Comparing KO vs WT
res <- results(dds, contrast=c("target_gt", "LRP1_HOMO", "LRP1_WT"))
#res <- lfcShrink(dds, coef="target_gtNRXN1_HOMO", type="apeglm")

# Summarize results
summary(res)

# MA-plot

plotMA(res, ylim=c(-2,2))

In [ ]:
#Copmare genotypes in a dosage-like manner

metadata$genotype_dosage <- as.numeric(factor(metadata$target_gt, levels=c("NRXN1_WT", "NRXN1_HET", "NRXN1_HOMO"))) - 1

dds <- DESeqDataSetFromMatrix(countData = counts, colData = metadata, design = ~ sex + genotype_dosage)

dds <- DESeq(dds)

res <- results(dds, name="genotype_dosage")

=summary(res)

In [ ]:
res_df <- as.data.frame(res)
res_df <- res_df[!is.na(res_df$padj),]
top_hits <- res_df[res_df$padj<0.05,]
top_hits[order(top_hits$padj),]
res_df["NRXN1",]
top_hits

In [ ]:
norm_counts <- counts(dds, normalized = TRUE)
head(norm_counts)

In [70]:
norm_counts_df <- as.data.frame(norm_counts)
norm_counts_df$gene <- row.names(norm_counts)
norm_counts_df <- norm_counts_df %>% 
  select(gene, everything())

In [ ]:
#distribution of LogFC values

plot_log2fc <- function(labeled_results, padj_threshold){
  filtered_results <- labeled_results %>% filter(padj < padj_threshold)
  p <- ggplot(filtered_results, aes(x = log2FoldChange)) +
    geom_histogram(binwidth = 0.1, fill = "royalblue1", color = 'black') +
    labs(title = "Histogram of Log2FoldChange for DE genes from DESeq2",
         x = "Log2FoldChange Values",
         y = "Count")
  print(p) 
}


log2fc <- plot_log2fc(merged_df, 0.05)

In [ ]:
#distribution of p-values
plot_pvals <- function(norm_counts) {
  p <- ggplot(norm_counts, aes(x = pvalue)) +
    geom_histogram(binwidth = 0.02, fill = "olivedrab3", color = 'black') +
    labs(title = "Histogram of Raw P-Values obtained from DE analysis",
         x = "P-Value",
         y = "Count")
  print (p)
}

pvals <- plot_pvals(merged_df)

In [ ]:
res_full <- as.data.frame(res)
res_full$gene <- rownames(res)

norm_counts_df <- as.data.frame(norm_counts)
norm_counts_df$gene <- rownames(norm_counts)

#merge norm_counts_df and res dfs
merged_df <- merge(res_full, norm_counts_df, by = "gene")
rownames(merged_df) <- merged_df$gene

merged_df <- merged_df[order(merged_df$padj, decreasing = FALSE), ]
#head(merged_df$gene, n=15)
top_genes <- head(merged_df$gene, n=14)

gene_vector <- paste0('"', top_genes, '"', collapse = ", ")

gene_vector <- gsub("'", "", gene_vector)

gene_list <- strsplit(gene_vector, ", ")[[1]]
gene_list <- trimws(gsub('"', '', gene_list))
gene_list <- c("NRXN1", gene_list)
gene_list


In [ ]:
#volcano plot
p1 <- EnhancedVolcano(merged_df,
    lab = merged_df$gene,
    selectLab = c('NRXN1'),
    x = 'log2FoldChange',
    y = 'padj',
    pCutoff = 0.05,
    FCcutoff = 1.0,
    col=c('black', 'black', 'RED3', 'red3'),
    #cutoffLineType = 'blank', 
    cutoffLineCol = 'grey',
    labSize = 4.0,
    labCol = 'black',
    labFace = 'bold',
    boxedLabels = TRUE,
    #boxedLabels = FALSE,
    pointSize = 1,
    colAlpha = 1,  
    legendPosition = 'right',
    legendLabSize = 14,
    legendIconSize = 4.0,
    drawConnectors = TRUE,
    widthConnectors = 1.0,
    colConnectors = 'grey',
    maxoverlapsConnectors = Inf,
    gridlines.major = FALSE,  
    gridlines.minor = FALSE
    #xlim = c(-6,6)
    #ylim = c(0,17) 
)

plot(p1)

In [ ]:
#visualizing NRXN1 expression

nrxn1_counts <- as.numeric(norm_counts["NRXN1", ])  
plot_df <- data.frame(sample = colnames(norm_counts), NRXN1 = nrxn1_counts)

plot_df <- plot_df %>%
  mutate(donor = metadata[match(sample, metadata$sample), "donor"],
         genotype = metadata[match(sample, metadata$sample), "genotype"])



plot_df$genotype <- factor(plot_df$genotype, levels = c("WT", "HET", "HOMO"))

# Plot NRXN1 expression with color coding by genotype
ggplot(plot_df, aes(x = genotype, y = NRXN1, color = genotype)) +
  geom_boxplot(alpha = 0.6) +  
  geom_point(size = 2, stroke = 0.5, color = "black") +  
  theme_minimal() +
  labs(title = "NRXN1 Expression by Genotype",
       x = "Genotype",
       y = "NRXN1 Counts") +
  theme(legend.position = "right", 
        axis.text.x = element_text(angle = 45, hjust = 1))  

In [ ]:
#NRXN1 expression per genotype and donor

plot_df <- data.frame(sample = colnames(norm_counts), NRXN1 = nrxn1_counts)

common_samples <- intersect(plot_df$sample, metadata$sample)

plot_df <- plot_df %>%
  filter(sample %in% common_samples) %>%
  mutate(donor = metadata[match(sample, metadata$sample), "donor"],
         genotype = metadata[match(sample, metadata$sample), "genotype"],
         PRS = metadata[match(sample, metadata$sample), "PRS"])


In [ ]:
library(ggbeeswarm) 
summary_df <- plot_df %>%
  group_by(donor, genotype)# %>%
  #summarise(NRXN1_avg = mean(NRXN1, na.rm = TRUE), .groups = 'drop')


genotypes <- c("WT", "HET", "HOMO")  
all_combinations <- expand.grid(donor = unique(plot_df$donor), genotype = genotypes)

complete_df <- left_join(all_combinations, summary_df, by = c("donor", "genotype"))

complete_df <- complete_df %>%
  replace_na(list(NRXN1 = 0)) %>%
  mutate(genotype = factor(genotype, levels = genotypes))

genotype_colors <- c(WT = "blue", HET = "purple", HOMO = "red")

avg_df <- complete_df %>%
  group_by(donor, genotype) %>%
  summarise(
    NRXN1_avg = mean(NRXN1, na.rm = TRUE),
    NRXN1_sem = sd(NRXN1, na.rm = TRUE) / sqrt(n()),  
    .groups = 'drop'
  )

# plotting NRXN1 expression per genotype and donor
g_avg_bar <- ggplot(avg_df, aes(x = donor, y = NRXN1_avg, fill = genotype)) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.7), width = 0.6) +
  geom_errorbar(aes(ymin = NRXN1_avg - NRXN1_sem, ymax = NRXN1_avg + NRXN1_sem), 
                position = position_dodge(width = 0.7), width = 0.25) +
  scale_fill_manual(values = genotype_colors) +
  theme_minimal() +
  labs(title = "Average NRXN1 Expression by Donor and Genotype",
       x = "Donor",
       y = "Average NRXN1 Expression") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5))

g_avg_bar


# GSEA Analysis

In [ ]:
library(fgsea)
library(forcats)

run_gsea_analysis <- function(donor_name, merged_results, gmt_paths) {
  
  # Function to create ranked log2 fold change vector
  make_ranked_log2fc <- function(merged_results) {
    rnk_vec <- deframe(merged_results[c("gene", "log2FoldChange")])
    rnk_vec <- rnk_vec[order(-rnk_vec)]
    return(rnk_vec)
  }
  
  ranked_logfc <- make_ranked_log2fc(merged_results)
  ranked_logfc_clean <- ranked_logfc[!is.na(ranked_logfc)]
  
  # Function to run FGSEA
  run_fgsea <- function(gmt_file_path, rnk_list, min_size, max_size) {
    fgsea_file <- fgsea::gmtPathways(gmt_file_path)
    fgsea_res <- fgsea(fgsea_file, rnk_list, minSize = min_size, maxSize = max_size)
    fgsea_res <- as_tibble(fgsea_res)
    return(fgsea_res)
  }
  
  # Initialize an empty list to store results
  fgsea_results <- list()
  
  # Iterate over the GMT file paths
  for (gmt_path in gmt_paths) {
    fgsea_res <- run_fgsea(gmt_path, ranked_logfc_clean, 15, 500)
    fgsea_results[[gmt_path]] <- fgsea_res
  }
  
  # Process and save results for each GMT file
  for (gmt_path in names(fgsea_results)) {
    fgsea_res_filt <- filter(fgsea_results[[gmt_path]], padj <= 0.05)
    
    # Create a data frame for the filtered results
    fgsea_res_df <- as.data.frame(fgsea_res_filt)
    list_cols <- sapply(fgsea_res_df, is.list)
    
    # Transform list columns
    fgsea_res_df[list_cols] <- lapply(fgsea_res_df[list_cols], function(x) {
      sapply(x, function(l) paste(l, collapse=", "))  
    })
    
    # Save the results
    gmt_name <- sub("^.*?/data/(.*)\\.gmt$", "\\1", gmt_path)
    output_file <- paste0("/stanley/nehme_lab/sbolshak/analysis/village_editing/GSEA/NRXN1/", donor_name, "_lrp1_mouse_final_", gmt_name, ".tsv")
    write_tsv(fgsea_res_df, output_file)
    
    # Create plots
    top_pathways <- function(fgsea_results, num_paths, padj_threshold) {
      fgsea_results <- fgsea_results %>%
        filter(padj <= padj_threshold) %>%
        mutate(negLogPadj = -log10(padj))
      
      fgsea_top <- fgsea_results %>%
        arrange(desc(abs(NES))) %>%
        slice_head(n = num_paths)
      
      p <- ggplot(fgsea_top, aes(x = NES, y = reorder(pathway, order(NES, decreasing = FALSE)))) +
        geom_point(aes(color = -log10(padj), size = size)) +
        labs(title = paste(donor_name, "GSEA results:", gmt_name),
             y = "Pathway", x = "NES") +
        theme_minimal()
      
      return(p)
    }
    
    plot <- top_pathways(fgsea_res_filt, 30, 0.05)
    ggsave(filename = paste0("/stanley/nehme_lab/sbolshak/analysis/village_editing/GSEA/NRXN1/", donor_name, "_lrp1_mouse_final_", gmt_name, ".png"), plot = plot, width = 12, height = 6, bg = "white")
  }
}

donor_name <- "all_samples"
gmt_files <- c('/stanley/nehme_lab/sbolshak/analysis/village_editing/data/h.all.v2024.1.Hs.symbols.gmt',
               '/stanley/nehme_lab/sbolshak/analysis/village_editing/data/c2.cp.v2024.1.Hs.symbols.gmt',
               '/stanley/nehme_lab/sbolshak/analysis/village_editing/data/c5.go.v7.4.syngo.bp.cc.v1.1.symbols.humgenSCZ.SCHEMA1.and.GWAS.1-2.gmt')

run_gsea_analysis(donor_name, merged_df, gmt_files)

# Run TRADE

In [ ]:
counts <- read.table("counts_matrix.txt", sep = '\t', header = TRUE)
counts[is.na(counts)] <- 0
rownames(counts) <- counts$GENE
counts <- counts[-1]
colnames(counts) <- gsub("\\.", "", colnames(counts))
head(counts)

In [ ]:
# first run differential expression on data using DEseq2
library(DESeq2,verbose=F)

coldata <- data.frame(row.names=colnames(counts))
coldata$target_gt <- as.factor(metadata[match(rownames(coldata),metadata$sample),"target_gt"])
# coldata$plate <- as.factor(metadata[match(rownames(coldata),metadata$sample),"plate"])
coldata$PRS <- as.factor(metadata[match(rownames(coldata),metadata$sample),"PRS"])
coldata$donor <- as.factor(metadata[match(rownames(coldata),metadata$sample),"donor"])
coldata$group <- paste0(coldata$donor,".",coldata$target_gt)


# Create a DESeqDataSet
dds <- DESeqDataSetFromMatrix(countData = counts,
                              colData = coldata,
                              design = ~0 + group)

In [ ]:
# filter dataset to only use genes with >10 counts in >=3 samples
smallestGroupSize <- 3
keep <- rowSums(counts(dds) >= 50) >= smallestGroupSize
table(keep)
dds <- dds[keep,]

In [ ]:
dds <- DESeq(dds) 
#res <- results(dds,contrast=c("target_gt","NRXN1_HOMO","NRXN1_WT"))
#res

In [ ]:
resultsNames(dds)

In [ ]:
library(data.table)
dir.create("/TRADE/NRXN1/")
targets_list <- unique(coldata[! coldata$target_gt %in% c("NRXN1_WT","LRP1_WT"),"group"])
for (target in targets_list) {
    if (grepl("NRXN1", target) & !grepl("WT", target)) {
    donor <- sub("\\..*", "", target)
    if (! paste0("group",donor,".NRXN1_WT") %in% resultsNames(dds)) { 
        next 
    } else {
        print(target)
        res_tmp <- results(dds,contrast=c("group",target,paste0(donor,".NRXN1_WT")))
    }
} else if (grepl("LRP1", target) & !grepl("WT", target)) {
    donor <- sub("\\..*", "", target)
    if (! paste0("group",donor,".LRP1_WT") %in% resultsNames(dds)) { 
        next 
    } else {
        print(target)
        res_tmp <- results(dds,contrast=c("group",target,paste0(donor,".LRP1_WT")))
    }
    } else {
        if (! paste0("group",donor,".NRXN1_WT") %in% resultsNames(dds)) { 
    message("Skipping: contrast group",donor,".NRXN1_WT not found")
    next 
}
        next
    }
    res_tmp_df <- as.data.frame(res_tmp)
    res_tmp_df$GENE <- rownames(res_tmp_df)
    # break
    write.table(as.data.frame(res_tmp_df),paste0("/TRADE/NRXN1/",target,"_vs_CTRL.results.txt"),sep="\t",quote=F,row.names=F,col.names=T)
}
print("Done!")

In [ ]:
targets_list

In [ ]:
library(mashr)
library(ggplot2)
library(ashr)
library(TRADE)
targets_list <- unique(coldata[! coldata$target_gt %in% c("NRXN1_WT","LRP1_WT"),"group"])
deseq_res_files <- list.files("/TRADE/NRXN1/")
# run TRADE on ecah set of differential expression results, then write the combined results to a new summary file
TRADE_out_df <- data.frame(row.names=c("mean","transcriptome_wide_impact","Me","KD"))
for (target in targets_list) {
    print(target)
    if (target %like% "NRXN1" & !target %like% "WT") {
        print("NRXN1...")
        donor <- sub("\\..*", "", target)
        if (! paste0(target,"_vs_CTRL.results.txt") %in% deseq_res_files) { 
            next 
        } else {
            print(target)
            res <- read.table(paste0("/TRADE/NRXN1/",target,"_vs_CTRL.results.txt"),sep="\t",header=T)
            TRADE_tmp <- TRADE(mode="univariate",results1=res)
            TRADE_out_df[c("mean","transcriptome_wide_impact","Me"),target] <- as.numeric(TRADE_tmp$distribution_summary)
            TRADE_out_df["KD",target] <- ifelse(res[res$GENE=="NRXN1",'padj']<0.05,TRUE,FALSE)
        }
    } else if (target %like% "LRP1" & !target %like% "WT") {
        print("LRP1...")
        donor <- sub("\\..*", "", target)
        if (! paste0(target,"_vs_CTRL.results.txt") %in% deseq_res_files){ 
            next 
        } else {
            res <- read.table(paste0("/TRADE/NRXN1/",target,"_vs_CTRL.results.txt"),sep="\t",header=T)
            TRADE_tmp <- TRADE(mode="univariate",results1=res)
            TRADE_out_df[c("mean","transcriptome_wide_impact","Me"),target] <- as.numeric(TRADE_tmp$distribution_summary)
            TRADE_out_df["KD",target] <- ifelse(res[res$GENE=="LRP1",'padj']<0.05,TRUE,FALSE)
        }
    } else {
        next
    }
    print(target)
}

TRADE_out_df <- as.data.frame(t(TRADE_out_df))
TRADE_out_df$target <- rownames(TRADE_out_df)
TRADE_out_df


In [ ]:
targets_list <- unique(coldata[! coldata$target_gt %in% c("NRXN1_WT","LRP1_WT"),"group"])
deseq_res_files <- list.files("/TRADE/NRXN1/") 
deseq_res_files

In [ ]:
TRADE_out_df$donor <- sub("\\..*", "", TRADE_out_df$target)
TRADE_out_df

In [ ]:
metadata_distinct <- metadata[!duplicated(metadata$donor), c("donor", "PRS", "sex", "genotype")]

# merge with TRADE_out_df
TRADE_out_df <- merge(TRADE_out_df, metadata_distinct, by = "donor", all.x = TRUE)
TRADE_out_df <- TRADE_out_df %>%
  select(-ends_with(".y"))
TRADE_out_df

In [ ]:
TRADE_out_df[TRADE_out_df$KD==1,]

In [ ]:
#plot TWI by genotype
options(repr.plot.width=12,repr.plot.height=6)
ggplot(TRADE_out_df,aes(x=genotype,y=transcriptome_wide_impact)) +
    geom_bar(aes(fill=target),position="dodge",stat="identity") +
    labs(y="TWI",fill="Donor",title="NRXN1") +
    theme(plot.title = element_text(hjust=0.5)) +
    geom_text(aes(label = ifelse(KD, "*", ""), group = target),position = position_dodge(width = 0.9), vjust = -.1, size = 20 / .pt)

In [ ]:
ggplot(TRADE_out_df,aes(x=log10(Me),y=log10(transcriptome_wide_impact),color=genotype)) +
    geom_point(size=4) +
    labs(x="# Effective Differentially Expressed Genes (log10)", y="TWI (log10)")

In [ ]:
library(ggpubr)

#plot average TWI for each genotype / PRS / sex

comparisons_list <- list(#c('HIGH', 'LOW'), c('HIGH', 'NEUTRAL'), c('LOW', 'NEUTRAL'))
                      c('HET', 'HOMO'))
                      #c('Female', 'Male'))

avg_data <- TRADE_out_df %>%
  group_by(genotype) %>%
  summarize(avg_TWI = mean(transcriptome_wide_impact, na.rm = TRUE), .groups = 'drop')


p <- ggplot(TRADE_out_df, aes(x = genotype, y = transcriptome_wide_impact, fill = genotype)) +
  geom_boxplot() +
  geom_jitter(width = 0.2, alpha = 0.5) +
  labs(y = "Transcriptome Wide Impact (TWI)", fill = "Genotype", title = "NRXN1 Average Trade Results by Genotype") +
  theme(plot.title = element_text(hjust = 0.5))


p + geom_signif(comparisons = comparisons_list, 
                 map_signif_level = TRUE, 
                 y_position = c(1, 1.2, 1.4))